# Importing the library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz,process


C:\Users\k050kgu\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Importing the excel

In [2]:
df= pd.read_excel('MatchPairs.xlsx', sheet_name='Sheet2')
df = df.replace(np.nan, '', regex=True)
df.head(2)

,PARTY_ID_1,FIRSTNAME_1,LASTNAME_1,MOBILE_1,PRIVATE_1,WORK_1,ELECTRONIC_ADDRESS_1,ST_ADDRESS_LINE_1,ST_CITY_1,ST_POSTAL_CODE_1,...,FIRSTNAME_2,LASTNAME_2,MOBILE_2,PRIVATE_2,WORK_2,ELECTRONIC_ADDRESS_2,ST_ADDRESS_LINE_2,ST_CITY_2,ST_POSTAL_CODE_2,Match
0,10030517,LISS,ØSTIGÅRD,93866445,9.38664e+07,9.38664e+07,$$$$$$,,MIDSUND,6475,...,LISS,RAKVÅG ØSTIGÅRD,93866445,9.38664e+07,9.38664e+07,BLOMSTERBUTIKKEN.LISS@GMAIL.COM,STØLHAUGVEGEN 3,MIDSUND,6475,1
1,10113433,MARIUS,FAALE,93268609,,,$$$$$$,,,,...,MARIUS,FAALE,93268609,,,§§§§§§,,,,1


# Levenshtein Distance FUnction

In [3]:
def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
  
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y
        
    

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    return (matrix[size_x - 1, size_y - 1])

In [4]:
print(levenshtein("oldKundan","Kundanold"))

6.0


In [5]:
def sorted_levenshtein(sequence1, sequence2):
    product1 = ''.join(sorted(sequence1))
    product2 = ''.join(sorted(sequence2))
    distance = levenshtein(product1, product2)
    return distance

def levenshtein_rate(product1, product2):
    distance = levenshtein(product1, product2)
    max_len = max(len(product1), len(product2))
    return 1 - (distance / max_len)

def sorted_levenshtein_rate(seq1, seq2):
    product1 = ''.join(sorted(seq1))
    product2 = ''.join(sorted(seq2))
    distance = levenshtein(product1, product2)
    max_len = max(len(product1), len(product2))
    return 1-(distance/max_len)

In [6]:
matchScore = pd.DataFrame([])

In [7]:
for index,row in df.iterrows():
    matchScore = matchScore.append(pd.DataFrame({
             'partyid1':row['PARTY_ID_1'],
             'partyid2':row['PARTY_ID_2'],
             #'full_name1':row['FULL_NAME_1'],
             #'full_name2':row['FULL_NAME_2'],
             #'nameScore':fuzz.token_set_ratio(str(row['FULL_NAME_1']).upper(),str(row['FULL_NAME_2']).upper())/100 ,
             'first_name1':row['FIRSTNAME_1'],
             'first_name2':row['FIRSTNAME_2'],
             'firstnameScore':fuzz.token_set_ratio(str(row['FIRSTNAME_1']).upper(),str(row['FIRSTNAME_2']).upper())/100 ,
            #'nameScore':sorted_levenshtein_rate(str(row['FULL_NAME_1']).upper(),str(row['FULL_NAME_2']).upper()) ,
             'last_name1':row['LASTNAME_1'],
             'last_name2':row['LASTNAME_2'],
             'lastnameScore':fuzz.token_set_ratio(str(row['LASTNAME_1']).upper(),str(row['LASTNAME_2']).upper())/100 ,
             'mobile1':row['MOBILE_1'],
             'mobile2':row['MOBILE_2'],
             'mobileScore': fuzz.ratio(str(row['MOBILE_1']),str(row['MOBILE_2']))/100, 
             'private1':row['PRIVATE_1'],
             'private2':row['PRIVATE_2'],
             'privateScore': fuzz.ratio(str(row['PRIVATE_1']),str(row['PRIVATE_2']))/100, 
             'work1':row['WORK_1'],
             'work2':row['WORK_2'],
             'workScore': fuzz.ratio(str(row['WORK_1']),str(row['WORK_2']))/100, 
             'email1':row['ELECTRONIC_ADDRESS_1'],
             'email2':row['ELECTRONIC_ADDRESS_2'],
             'emailScore': fuzz.partial_ratio(str(row['ELECTRONIC_ADDRESS_1']).upper(),str(row['ELECTRONIC_ADDRESS_2']).upper())/100, 
             'addressLine1':row['ST_ADDRESS_LINE_1'],
             'addressLine2':row['ST_ADDRESS_LINE_2'],
             
             'addressLineScore': fuzz.token_sort_ratio(str(row['ST_ADDRESS_LINE_1']).upper(),str(row['ST_ADDRESS_LINE_2']).upper())/100,
             'city1':row['ST_CITY_1'],
             'city2':row['ST_CITY_2'],
             'cityScore': fuzz.partial_ratio(str(row['ST_CITY_1']).upper(),str(row['ST_CITY_2']).upper())/100,
             'post1':row['ST_POSTAL_CODE_1'],
             'post2':row['ST_POSTAL_CODE_2'],
             'postScore':fuzz.ratio(str(row['ST_POSTAL_CODE_1']).upper(),str(row['ST_POSTAL_CODE_2']).upper())/100, 
              'Match':row['Match']
            },index=[0]), ignore_index=True)


In [8]:
matchScore.to_excel('MatchResults7.xlsx')

In [9]:
df_reg=matchScore[['firstnameScore','lastnameScore','mobileScore','privateScore','workScore','emailScore','addressLineScore','cityScore','postScore','Match']]
#df_regShow=matchScore
#df_reg=matchScore[['nameScore','mobileScore','privateScore','workScore','emailScore','addressLineScore','cityScore','postScore','Match']]


In [10]:
x = df_reg.iloc[:, :-1].values
y = df_reg.iloc[:, -1].values
y=y.astype('int')

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)

In [12]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 200, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [13]:
y_pred = classifier.predict(X_test)

In [14]:
#print(np.concatenate((X_test,y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))
#print(np.concatenate([y_pred,y_test], axis=0)
final = pd.DataFrame(np.concatenate((X_test,y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))
print(final)
final.to_excel('MatchResults6.xlsx')


        0     1    2    3    4     5     6     7     8    9   10
0    1.00  0.75  1.0  1.0  1.0  1.00  0.44  1.00  0.83  1.0  1.0
1    0.22  1.00  1.0  1.0  1.0  0.00  0.00  1.00  1.00  0.0  0.0
2    0.29  1.00  1.0  1.0  1.0  1.00  1.00  1.00  1.00  0.0  0.0
3    0.30  1.00  1.0  0.0  0.5  1.00  0.30  0.18  0.67  0.0  0.0
4    1.00  1.00  1.0  0.0  0.0  1.00  0.49  0.38  0.67  1.0  1.0
..    ...   ...  ...  ...  ...   ...   ...   ...   ...  ...  ...
114  0.25  0.20  1.0  0.0  1.0  0.00  0.31  0.31  0.67  0.0  0.0
115  1.00  1.00  1.0  0.0  1.0  1.00  1.00  0.00  0.00  1.0  1.0
116  1.00  1.00  1.0  0.0  0.0  0.00  0.00  0.00  0.00  1.0  1.0
117  0.09  0.33  1.0  1.0  1.0  1.00  0.00  1.00  1.00  0.0  0.0
118  0.34  0.20  1.0  0.4  0.0  0.53  0.64  1.00  1.00  0.0  0.0

[119 rows x 11 columns]


In [15]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[59  2]
 [ 5 53]]
              precision    recall  f1-score   support

           0       0.92      0.97      0.94        61
           1       0.96      0.91      0.94        58

    accuracy                           0.94       119
   macro avg       0.94      0.94      0.94       119
weighted avg       0.94      0.94      0.94       119

0.9411764705882353


In [16]:
#print(fuzz.partial_ratio('OLAV','SILJE'))
#print(fuzz.token_set_ratio('OLAV','SILJE'))
print(fuzz.token_sort_ratio("nan","STØLHAUGVEGEN 3"))
#print(fuzz.ratio('OLAV','SILJE'))
print(sorted_levenshtein_rate('','SILJE'))

12
0.0


In [17]:
print(classifier.predict([[fuzz.token_set_ratio('IGOR'.upper(),'IGOR'.upper())/100 ,
                           fuzz.token_set_ratio('KICA'.upper(),'KICA'.upper())/100 ,
                           fuzz.ratio('96824370','96824370')/100, 
                           fuzz.ratio('','')/100, 
                           fuzz.ratio('96824370','')/100, 
                           fuzz.partial_ratio('IGOR.KICA@GMAIL.COM'.upper(),'IGOR.KICA@GMAIL.COM'.upper())/100, 
                           fuzz.token_sort_ratio('LØKKAVEGEN 11 C'.upper(),'RISKOLLVEIEN 27'.upper())/100,
                           fuzz.partial_ratio('JESSHEIM'.upper(),'ØSTERÅS'.upper())/100,
                           fuzz.ratio('2052'.upper(),'1332'.upper())/100]]))

[1]


In [18]:
testDf= pd.read_excel('TestDataSet1.xlsx', sheet_name='Sheet3')
testDf = testDf.replace(np.nan, '', regex=True)

testPrediction = pd.DataFrame([])
testPrediction["Match"]= ""
testPrediction.head(2)

,Match


In [19]:
for index,row in testDf.iterrows():
  
     #print(row['PARTY_ID_1'])
     testPrediction = testPrediction.append(pd.DataFrame({
             
             'partyid1':row['PARTY_ID_1'],
             'partyid2':row['PARTY_ID_2'],
             #'full_name1':row['FULL_NAME_1'],
             #'full_name2':row['FULL_NAME_2'],
             #'nameScore':fuzz.token_set_ratio(str(row['FULL_NAME_1']).upper(),str(row['FULL_NAME_2']).upper())/100 ,
             'first_name1':row['FIRSTNAME_1'],
             'first_name2':row['FIRSTNAME_2'],
             'firstnameScore':fuzz.token_set_ratio(str(row['FIRSTNAME_1']).upper(),str(row['FIRSTNAME_2']).upper())/100 ,
            #'nameScore':sorted_levenshtein_rate(str(row['FULL_NAME_1']).upper(),str(row['FULL_NAME_2']).upper()) ,
             'last_name1':row['LASTNAME_1'],
             'last_name2':row['LASTNAME_2'],
             'lastnameScore':fuzz.token_set_ratio(str(row['LASTNAME_1']).upper(),str(row['LASTNAME_2']).upper())/100 ,
             'mobile1':row['MOBILE_1'],
             'mobile2':row['MOBILE_2'],
             'mobileScore': fuzz.ratio(str(row['MOBILE_1']),str(row['MOBILE_2']))/100, 
             'private1':row['PRIVATE_1'],
             'private2':row['PRIVATE_2'],
             'privateScore': fuzz.ratio(str(row['PRIVATE_1']),str(row['PRIVATE_2']))/100, 
             'work1':row['WORK_1'],
             'work2':row['WORK_2'],
             'workScore': fuzz.ratio(str(row['WORK_1']),str(row['WORK_2']))/100, 
             'email1':row['ELECTRONIC_ADDRESS_1'],
             'email2':row['ELECTRONIC_ADDRESS_2'],
             'emailScore': fuzz.partial_ratio(str(row['ELECTRONIC_ADDRESS_1']).upper(),str(row['ELECTRONIC_ADDRESS_2']).upper())/100, 
             'addressLine1':row['ST_ADDRESS_LINE_1'],
             'addressLine2':row['ST_ADDRESS_LINE_2'],
             
             'addressLineScore': fuzz.token_sort_ratio(str(row['ST_ADDRESS_LINE_1']).upper(),str(row['ST_ADDRESS_LINE_2']).upper())/100,
             'city1':row['ST_CITY_1'],
             'city2':row['ST_CITY_2'],
             'cityScore': fuzz.partial_ratio(str(row['ST_CITY_1']).upper(),str(row['ST_CITY_2']).upper())/100,
             'post1':row['ST_POSTAL_CODE_1'],
             'post2':row['ST_POSTAL_CODE_2'],
             'postScore':fuzz.ratio(str(row['ST_POSTAL_CODE_1']).upper(),str(row['ST_POSTAL_CODE_2']).upper())/100, 
             'Match':classifier.predict([[
                           fuzz.token_set_ratio(str(row['FIRSTNAME_1']).upper(),str(row['FIRSTNAME_2']).upper())/100 ,
                           fuzz.token_set_ratio(str(row['LASTNAME_1']).upper(),str(row['LASTNAME_2']).upper())/100 ,
                           fuzz.ratio(str(row['MOBILE_1']),str(row['MOBILE_2']))/100,  
                           fuzz.ratio(str(row['PRIVATE_1']),str(row['PRIVATE_2']))/100, 
                           fuzz.ratio(str(row['WORK_1']),str(row['WORK_2']))/100, 
                           fuzz.partial_ratio(str(row['ELECTRONIC_ADDRESS_1']).upper(),str(row['ELECTRONIC_ADDRESS_2']).upper())/100, 
                           fuzz.token_sort_ratio(str(row['ST_ADDRESS_LINE_1']).upper(),str(row['ST_ADDRESS_LINE_2']).upper())/100,
                           fuzz.partial_ratio(str(row['ST_CITY_1']).upper(),str(row['ST_CITY_2']).upper())/100,
                           fuzz.ratio(str(row['ST_POSTAL_CODE_1']).upper(),str(row['ST_POSTAL_CODE_2']).upper())/100
                              ]])
            },index=[0]), ignore_index=True)
    

In [20]:
testPrediction.to_excel('TestSetPrediction.xlsx')